# Подключение

In [1]:
import pyodbc

In [2]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.199.13.60;DATABASE=rway;UID=vkomarnitskii;PWD=Rway1')
cursor = conn.cursor()


## Pandas

In [3]:
import pandas as pd
from math import isnan
from numpy import float64
from pprint import pprint

Тут можно установить параметры отображения для pandas. Сколько показывать строк и столбцов соответственно.

In [4]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 150

### Отладка


In [5]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции
    """
    import time
    
    def wrapper(*args, **kwargs):
        t = time.time()
        res = func(*args, **kwargs)
        print(f'{func.__name__} выполнилась за время {time.time() - t:.2f}s')
        return res
    
    return wrapper

## Необходимые функции

Позволяет получить список характеристик для всех предложений из задачи `task_id`. 

In [6]:
@benchmark
def get_harks_by_object_keys(task_id):
    """
    Позволяет получить список характеристик для всех предложений из задачи task_id. 
    :param task_id: id задачи
    :type task_id: str
    :return: DataFrame, в котором указаны характеристики и их значения для всех объектов из задачи task_id
    """
    
#     in_expr = f'({", ".join(list_of_keys)})'
    query = '''
    SELECT
    t_har.Наименование,
    CASE 
        WHEN t.Значение_Тип = 04 THEN CAST(t.Значение_Дата AS varchar)
        WHEN t.Значение_Тип = 03 THEN CAST(t.Значение_Число AS varchar)
        WHEN t.Значение_Тип = 05 THEN t.Значение_Строка
        WHEN t.Значение_Тип = 08 THEN 
        COALESCE(
            (SELECT Наименование FROM [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop WHERE t_dop.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types WHERE t_types.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыСделки] t_types_1 WHERE t_types_1.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ЛогическиеЗначения] t_logic WHERE t_logic.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыВерифицированности] t_verif WHERE t_verif.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ПодСегменты] t_podseg WHERE t_podseg.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[Застройщики] t_zastr WHERE t_zastr.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations WHERE t_stations.Ссылка = t.Значение)                  
        )
    END
        AS Значение,
    t.Объект AS Ссылка
FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
    INNER JOIN [rway].[ПВХ].[Характеристики] t_har
        ON t_har.[Ссылка] = t.[Характеристика]
    INNER JOIN [rway].[РегистрСведений].[ПредложенияЗадач] reg_task
        ON reg_task.Предложение = t.Объект
        AND reg_task.Задача in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{}')
    '''.format(task_id)
    
    cursor.execute(query)
#     data_df = pd.read_sql(sql=query, con=conn)

    # Преобразование данных, которые вернул запрос в список списков, одновременно все байты преобразуются в hex
    data = list(map(lambda x: list(x), cursor.fetchall()))
    data_df = pd.DataFrame(data, columns=list(map(lambda x: x[0], cursor.description)))
    # Создание словаря для DataFrame
    return data_df.pivot(index='Ссылка', columns='Наименование', values='Значение')

    

Функция позволяет получить все предложения по конкретному заданию. Задание указывается в виде строки id задания.

Например, `'0001-0405'`

In [7]:
@benchmark
def get_base_info_by_task_id(task_id, offers_count=None):
    """
    :param task_id: id задания
    :param offers_count: Количество предложений для выбора из базы (пустое для выбора всех предложений)
    :type task_id: str
    :return: DataFrame, содержащий все поля, которые можно получить из таблицы "ПредложенияЗадач"
    """
    
    # Запрос к дате задания
    date = cursor.execute("SELECT Дата FROM [rway].[Документ].[Задание] WHERE КодЗадания = '{}'".format(task_id)).fetchone()
    
    top = '' if not offers_count else 'TOP {}'.format(offers_count)
    query = '''
    SELECT {top}
          t.Код,
          t.Ссылка,
          t_task._Fld198 AS КодЗадачи,
          t_source.Наименование AS Источник,
          t.АдресAhunter,
          t.АктуальнаяСсылкаИсточника,
          t.ДатаПересмотраЭкспозиции,
          t.ДатаПроверкиАктуальности,
          t.ДатаРазмещения,
          t.Город,
          t.Описание,
          (SELECT Наименование FROM [rway].[Справочник].[Подсегменты] WHERE Ссылка = t.Подсегмент) AS Подсегмент,
          (SELECT Наименование FROM [rway].[Справочник].[Сегменты] WHERE Ссылка = t.Сегмент) AS Сегмент,
          (SELECT Наименование FROM [rway].[Справочник].[СубъектыРФ] WHERE Ссылка = t.Субъект) AS Субъект,
          t.СсылкаИсточника,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыРынка] WHERE Ссылка = t.ТипРынка) AS ТипРынка,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыСделки] WHERE Ссылка = t.ТипСделки) AS ТипСделки
    FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
        JOIN [rway].[РегистрСведений].[ПредложенияЗадач] t_offer
            ON Ссылка = t_offer.[Предложение]
        JOIN [rway].[dbo].[_Task62] t_task
            ON t_task.[_IDRRef] = t_offer.[Задача] AND t_task.[_IDRRef] in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{task_id}')
        JOIN [rway].[Справочник].[Источники] t_source
            ON t_source.Ссылка = ИсточникИнформации

    '''.format(top=top, task_id=task_id)
    
    cursor.execute(query)    
    data = cursor.fetchall()
    # Те же самые преобрзования, как в функции с характеристиками
    df = pd.DataFrame(map(lambda x: list(x), data))
    df.columns = list(map(lambda x: x[0].replace('.', '_'), cursor.description))
    df['КодЗадания'] = task_id
    df['ДатаЗадания'] = date[0]
#     df = pd.read_sql(sql=query, con=conn)
    return df
    

# Боевой запуск
Здесь необходимо указать код задания, по которому будет происходить сбор данных.

In [8]:
task_id = '0001-0405'

In [9]:
# Сбор основных данных из "ПредложенияОбъектовНедвижимости"
base_df = get_base_info_by_task_id(task_id)

KeyboardInterrupt: 

Вывести таблицу с базовыми характеристиками

In [ ]:
base_df

Сбор характеристик по ссылкам из основного DataFrame base_df

In [ ]:
har_df = get_harks_by_object_keys(task_id)

Вывести таблицу со значениями харакетристик из ПВХ.

In [ ]:
har_df

Собираем обе таблицы в одну большую

In [ ]:
result_df = pd.merge(base_df, har_df, on='Ссылка', how='left')

In [ ]:
result_df

Функция, в которой и происходит тестирование.

In [ ]:
@benchmark
def make_test():
    new_df = pd.DataFrame(columns=set(result_df.loc[:, 'Источник']), index=list(result_df.columns.fillna(0)) + ['Всего'])
    nan = float('nan')
    for source in set(result_df['Источник']):
        filtered = result_df[result_df['Источник'] == source]
        new_df.loc['Всего', source] = len(filtered)
        count_s = filtered.replace('', nan).replace('НетДанных', nan).replace('0.000000', nan).count()
        for k, v in count_s.items():
            new_df.loc[k, source] = v * 100 / new_df.loc['Всего', source]
        new_df.loc['КодЗадания', source] = list(filtered['КодЗадания'])[0]
        new_df.loc['КодЗадачи', source] = list(filtered['КодЗадачи'])[0]
    new_df.loc['ДатаЗадания'] = list(filtered['ДатаЗадания'])[0]
    new_df = new_df.T

    return new_df

In [ ]:
test = make_test()

Вывести на экран таблицу с результатами теста

In [ ]:
test

Выгрузка данных в `csv` файл для удобного просмотра в Excel

In [ ]:
# test.to_csv('test_result.csv', header=list(test.columns), encoding='cp1251', sep=';')
test.columns = list(map(lambda x: x.replace('(', '_').replace(')', '_').replace('.', '_').replace(' ', '_'), test.columns))

Запись результатов теста в базу

In [ ]:
test.to_sql('fill_rate', con=engine, if_exists='append')